In [1]:
#기본 : 콜옵션 - 가격이 상승할 것이라 예상 (수익률 하락), 풋옵션 - 가격이 하락할 것이라 예상 (수익률 상승)

In [153]:
import numpy as np
import pandas as pd
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)
from scipy.stats import norm
from pykrx import stock

In [196]:
import pandas as pd
import numpy as np
from scipy.stats import norm

def black_scholes_kr(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        option_price = (S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2))
    elif option_type == 'put':
        option_price = (K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1))
    else:
        raise ValueError("Option type must be 'call' or 'put'")
    
    return option_price

file_path = 'data_3030_20240508.csv'
df = pd.read_csv(file_path, encoding='EUC-KR')

print(df.head())

S = 375.30   # 선물 현재가
K = 495   # 행사가
T = 0.25  
r = 0.035  # 한국은행 기준금리
sigma = 0.1410

call_price = black_scholes_kr(S, K, T, r, sigma, option_type='call')
put_price = black_scholes_kr(S, K, T, r, sigma, option_type='put')

print(f"Calculated Call Option Price: {call_price:.2f}")
print(f"Calculated Put Option Price: {put_price:.2f}")

   CALL_EUREX 정산가  CALL_거래량  CALL_매도  CALL_매수  CALL_대비  CALL_현재가    행사가  \
0            0.01         0      5.3      0.0      0.0       0.0  495.0   
1            0.01         0      0.0      0.0      0.0       0.0  492.5   
2            0.01         0      0.0      0.0      0.0       0.0  490.0   
3            0.01         0      0.0      0.0      0.0       0.0  487.5   
4            0.01         0      0.0      0.0      0.0       0.0  485.0   

   PUT_현재가  PUT_대비  PUT_매수  PUT_매도  PUT_거래량  PUT_EUREX 정산가  
0      0.0     0.0     0.0     0.0        0         120.15  
1      0.0     0.0     0.0     0.0        0         117.65  
2      0.0     0.0     0.0     0.0        0         115.15  
3      0.0     0.0     0.0     0.0        0         112.70  
4      0.0     0.0     0.0     0.0        0         110.20  
Calculated Call Option Price: 0.00
Calculated Put Option Price: 115.39


- C는 콜 옵션 가격, 
- P는 풋 옵션 가격, 
- S는 현재 주식 가격,
- K는 행사 가격,
- r는 무위험 이자율,
- T는 만기까지의 시간,
- t는 현재 시간을 의미함

- 한국의 경우, 반도체 산업에 영향을 크게 받고 있다는 것을 반영함 
- 환율의 영향을 함께 넣음

In [155]:
#091160
df = stock.get_etf_ohlcv_by_date("20210101", "20240508", "091160", 'd')
print(df.head())

                 NAV     시가     고가     저가     종가     거래량         거래대금     기초지수
날짜                                                                            
2021-01-04  34516.76  34145  34450  33680  34355  425277  14530310755  3526.35
2021-01-05  34802.98  34195  34710  34000  34710  310443  10696441785  3555.88
2021-01-06  34785.19  35200  35705  34515  34670  378577  13286465390  3554.07
2021-01-07  35244.13  35330  35515  35000  35225  228026   8046457615  3601.41
2021-01-08  35621.34  35725  35725  34975  35480  458312  16138424380  3640.44


In [156]:
df.corr().iplot(kind='heatmap', colorscale='Blues')

In [157]:
temp = df[['기초지수','종가']].copy()
temp.head()

,기초지수,종가
날짜,,
2021-01-04,3526.35,34355
2021-01-05,3555.88,34710
2021-01-06,3554.07,34670
2021-01-07,3601.41,35225
2021-01-08,3640.44,35480


In [158]:
gdp = pd.read_csv('kr_gdp.csv')
gdp = gdp[72:]
gdp.head()

,Unnamed: 0,기간,Unnamed: 1,Unnamed: 2,Trend,Cycle
72,249,2021-Q1,1.8,2.4,0.540053,1.259947
73,250,2021-Q2,0.9,6.4,0.547993,0.352007
74,251,2021-Q3,0.1,4.1,0.555523,-0.455523
75,252,2021-Q4,1.4,4.3,0.562824,0.837176
76,253,2022-Q1,0.7,3.1,0.569795,0.130205


In [159]:
q_temp = temp.resample('Q').mean()
q_temp.index = q_temp.index.to_period('Q')
q_temp['종가'] = (q_temp['종가']) / 10
q_temp

,기초지수,종가
날짜,,
2021Q1,3687.404000,3600.333333
2021Q2,3872.842222,3769.658730
2021Q3,3763.981452,3657.241935
2021Q4,3611.307460,3501.190476
2022Q1,3723.561695,3640.754237
2022Q2,3463.170484,3376.500000
2022Q3,2743.950952,2663.587302
2022Q4,2501.629839,2408.411290
2023Q1,2621.710000,2546.548387


In [160]:
gdp['기간'] = pd.to_datetime(gdp['기간'])
gdp.set_index('기간', inplace=True)

gdp.index = gdp.index.to_period('Q').astype(str).str.replace('-', '')
gdp

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Trend,Cycle
기간,,,,,
2021Q1,249,1.8,2.4,0.540053,1.259947
2021Q2,250,0.9,6.4,0.547993,0.352007
2021Q3,251,0.1,4.1,0.555523,-0.455523
2021Q4,252,1.4,4.3,0.562824,0.837176
2022Q1,253,0.7,3.1,0.569795,0.130205
2022Q2,254,0.8,2.9,0.576856,0.223144
2022Q3,255,0.2,3.2,0.584510,-0.384510
2022Q4,256,-0.3,1.4,0.593400,-0.893400
2023Q1,257,0.3,0.9,0.603925,-0.303925


In [161]:
gdp.index

Index(['2021Q1', '2021Q2', '2021Q3', '2021Q4', '2022Q1', '2022Q2', '2022Q3',
       '2022Q4', '2023Q1', '2023Q2', '2023Q3', '2023Q4', '2024Q1'],
      dtype='object', name='기간')

In [162]:
#069500
df2 = stock.get_etf_ohlcv_by_date("20210101", "20240508", "069500", 'd')
print(df2.head())

                 NAV     시가     고가     저가     종가       거래량          거래대금  \
날짜                                                                         
2021-01-04  40615.17  39585  40615  39405  40540  12185235  489888903885   
2021-01-05  41230.17  40325  41150  40200  41150   9126676  370933192250   
2021-01-06  40835.69  41270  41680  40595  40750  15221599  625103057790   
2021-01-07  41709.04  41210  42040  41190  41700   8752407  364764290470   
2021-01-08  43651.13  42245  43785  42115  43545  11521190  494420112265   

              기초지수  
날짜                  
2021-01-04  399.88  
2021-01-05  406.03  
2021-01-06  402.08  
2021-01-07  410.81  
2021-01-08  430.22  


In [163]:
kospi = df2[['기초지수', '종가']]
kospi = kospi.resample('Q').mean()
kospi.index = kospi.index.to_period('Q')
kospi['종가'] = (kospi['종가']) / 100
kospi

,기초지수,종가
날짜,,
2021Q1,417.276167,423.270000
2021Q2,428.846984,430.760317
2021Q3,421.273548,421.629839
2021Q4,392.225079,392.611905
2022Q1,368.322542,372.559322
2022Q2,343.438871,345.176613
2022Q3,313.786825,314.438889
2022Q4,304.609839,305.306452
2023Q1,315.215484,319.387903


In [164]:
print(gdp['Cycle'].iplot(kind='scatter'))
print(q_temp.iplot(kind='scatter'))
print(kospi.iplot(kind='scatter'))

None


None


None


In [165]:
result = pd.merge(q_temp, kospi, on='날짜')
result

,기초지수_x,종가_x,기초지수_y,종가_y
날짜,,,,
2021Q1,3687.404000,3600.333333,417.276167,423.270000
2021Q2,3872.842222,3769.658730,428.846984,430.760317
2021Q3,3763.981452,3657.241935,421.273548,421.629839
2021Q4,3611.307460,3501.190476,392.225079,392.611905
2022Q1,3723.561695,3640.754237,368.322542,372.559322
2022Q2,3463.170484,3376.500000,343.438871,345.176613
2022Q3,2743.950952,2663.587302,313.786825,314.438889
2022Q4,2501.629839,2408.411290,304.609839,305.306452
2023Q1,2621.710000,2546.548387,315.215484,319.387903


In [166]:
result.columns

Index(['기초지수_x', '종가_x', '기초지수_y', '종가_y'], dtype='object')

In [167]:
result[['기초지수_x', '기초지수_y']].corr().iplot(kind='heatmap')

In [171]:
gdp.index = gdp.index.rename('날짜')
gdp = gdp[['Trend', 'Cycle']]
gdp

,Trend,Cycle
날짜,,
2021Q1,0.540053,1.259947
2021Q2,0.547993,0.352007
2021Q3,0.555523,-0.455523
2021Q4,0.562824,0.837176
2022Q1,0.569795,0.130205
2022Q2,0.576856,0.223144
2022Q3,0.584510,-0.384510
2022Q4,0.593400,-0.893400
2023Q1,0.603925,-0.303925


In [181]:
df = pd.read_csv('kr_gdp.csv')
df[['기간', 'Cycle']] #경기국면이 상승할 것이라 예상 가능함

,기간,Cycle
0,2003-Q1,-1.527653
1,2003-Q2,-1.068776
2,2003-Q3,0.991055
3,2003-Q4,1.653465
4,2004-Q1,0.319455
...,...,...
80,2023-Q1,-0.303925
81,2023-Q2,-0.015931
82,2023-Q3,-0.029070
83,2023-Q4,-0.042985


In [185]:
data = stock.get_etf_ohlcv_by_date("20240101", "20240508", "091160", 'd')
data['기초지수'].iplot(kind='scatter')

- 코스피 200 지수가 상승할 것이라 생각되므로, 콜옵션 구매
- 만기는 7월 (0.25)